# Polynomial Bases and Transformations

The Minterpy polynomials you've encountered so far have been represented in the Lagrange and Newton bases.
Minterpy supports a range of polynomial bases, each with a consistent interface, meaning that polynomials in different bases behave and can be manipulated in similar ways.

In this in-depth tutorial, you'll explore:

- various polynomial bases supported by Minterpy
- transformation between bases

For simplicity, this tutorial uses examples in one and two dimensions, but the principles apply to polynomials of higher dimensions as well.

Before diving in, make sure to import the necessary packages:

In [ ]:
import minterpy as mp
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Polynomials in Minterpy

Polynomials in Minterpy are multidimensional polynomials defined on $[-1, 1]^m$ where $m$ is the spatial dimension:

$$
Q(\boldsymbol{x}) = \sum_{\boldsymbol{\alpha} \in A} c_{\cdot, \boldsymbol{\alpha}} \Psi_{\cdot, \boldsymbol{\alpha}} (\boldsymbol{x}),
$$

where:

- $A$ is the multi-index set of the exponents, i.e., $\boldsymbol{\alpha} \in A \subseteq \mathbb{N}^m$ which generalizes the notion of polynomial degree to multiple dimensions.
- $\Psi_{\cdot, \boldsymbol{\alpha}}$ is the multidimensional basis polynomial associated with the multi-index element $\boldsymbol{\alpha}$, the placeholder $\cdot$ indicates that the basis polynomials depend on the chosen basis.
- $c_{\cdot, \boldsymbol{\alpha}}$ is the coefficient that corresponds to the multidimensional basis polynomial $\Psi_{\cdot, \boldsymbol{\alpha}}$.

Minterpy supports several polynomial bases as shown in the table below.

|            Basis            |                                       Constructor                                       |
|:---------------------------:|:---------------------------------------------------------------------------------------:|
|          Lagrange           |  {py:class}`LagrangePolynomial <.polynomials.lagrange_polynomial.LagrangePolynomial>`   |
|           Newton            |     {py:class}`NewtonPolynomial <.polynomials.newton_polynomial.NewtonPolynomial>`      |
|          Canonical          | {py:class}`CanonicalPolynomial <.polynomials.canonical_polynomial.CanonicalPolynomial>` |
| Chebyshev (of the 1st kind) | {py:class}`ChebyshevPolynomial <.polynomials.chebyshev_polynomial.ChebyshevPolynomial>` |


### Lagrange basis

The Lagrange basis was the first polynomial basis you encountered in this series of tutorials. In Minterpy, the multidimensional Lagrange basis polynomial associated with the multi-index element $\boldsymbol{\alpha}$ is defined as a polynomial that satisfies the following condition:

$$
\Psi_{\mathrm{lag}, \boldsymbol{\alpha}} (\boldsymbol{p}_{\boldsymbol{\beta}}) = \delta_{\boldsymbol{\alpha}, \boldsymbol{\beta}}, \;\; \boldsymbol{p}_{\boldsymbol{\beta}} \in P_{A}, \;\; \boldsymbol{\alpha}, \boldsymbol{\beta} \in A,
$$

where $P_{A}$ is the set of unisolvent nodes, $A$ is the multi-index set of exponents, and $\delta_{\cdot, \cdot}$ is the Kronecker delta.

```{note}
The condition above is necessary for defining the Lagrange basis polynomials, but it does not provide the full expression of these basis polynomials as functions of $\boldsymbol{x} \in [-1, 1]^m$.
Indeed in Minterpy, Lagrange basis polynomials are conceptual basis polynomials that can only be expressed in terms of another polynomial basis.
```

Polynomials in the Lagrange basis in Minterpy have limited functionality; for instance, they cannot be directly evaluated at a set of query points. Nevertheless, they serve as an entry point for function approximations using interpolating polynomials because their coefficients are highly intuitive.
Specifically, the coefficient corresponding to the multi-index element $\boldsymbol{\alpha}$ of an interpolating polynomial approximating a function $f$ is

$$
c_{\mathrm{lag}, \boldsymbol{\alpha}} = f(\boldsymbol{p}_{\boldsymbol{\alpha}}).
$$

In other words, the coefficients are simply the values of the function at the unisolvent nodes.

To do more with polynomials in the Lagrange basis, such as evaluation or manipulation, you need to transform them to another basis. This process will be covered in a later section of this tutorial.

### Newton basis

You've also seen the Newton basis by following these tutorials. The multidimensional Newton basis polynomial associated with multi-index element $\alpha$ is defined as:

$$
\Psi_{\mathrm{nwt}, \boldsymbol{\alpha}} (\boldsymbol{x}) = \prod_{j = 1}^m \prod_{i = 0}^{\alpha - 1} (x_j - p_{i, j}),
$$

where $p_{i, j}$'s are the interpolation points along dimension $j$ (i.e., the so-called {ref}`generating points / nodes <fundamentals/interpolation-at-unisolvent-nodes:Generating points>` which in multiple dimensions are not the same as the unisolvent nodes).

```{note}
As you can see in the above definition, the Newton basis polynomials depend on the generating points of an interpolation grid. Such points are an integral part of the definition of the basis.
```

As an example, consider the two-dimensional multi-index set:

$$
A = \{ (0, 0), (1, 0), (2, 0), (3, 0), (0, 1), (1, 1), (2, 1), (0, 2) \}.
$$

Newton basis polynomial can only be defined with respect to a given generating points. Valid generating points that are distributed according to the Leja-ordered Chebyshev-Lobatto points are given in the table below.

| $\mathrm{dim} = 1$ | $\mathrm{dim} = 2$ |
|:-------:|:-------:|
| $1.0$   | $-1.0$  |
| $-1.0$  | $1.0$   |
| $0.5$   | $-0.5$  |


The Newton basis polynomials according to the above multi-index set of exponents are given in the table below.

| $\boldsymbol{\alpha} \in A$ | $\Psi_{\mathrm{nwt}, \boldsymbol{\alpha} }$ |
|:---------:|:--------------------------------------:|
| $(0, 0)$  | $1.0$                                  | 
| $(1, 0)$  | $x_1 - 1.0$                            |
| $(2, 0)$  | $(x_1 - 1.0) (x_1 + 1.0)$              |
| $(3, 0)$  | $(x_1 - 1.0) (x_1 + 1.0) (x_1 - 0.5)$  |
| $(0, 1)$  | $x_2 + 1.0$                            |
| $(1, 1)$  | $(x_1 - 1.0) (x_2 + 1.0)$              |
| $(2, 1)$  | $(x_1 - 1.0) (x_1 + 1.0) (x_2 + 1.0)$  |
| $(0, 2)$  | $(x_2 + 1.0) (x_2 - 1.0)$              |

### Canonical basis

The canonical basis is arguably the most familiar polynomial basis of them all. The multidimensional canonical basis polynomial associated with the multi-index element $\boldsymbol{\alpha}$ is defined as:

$$
\Psi_{\mathrm{can}, \boldsymbol{\alpha}} (\boldsymbol{x}) = x_1^{\alpha_1} \cdots x_m^{\alpha_m} = \prod_{j = 1}^m x_j^{\alpha_j}.
$$

```{warning}
While polynomials in the canonical basis can be evaluated at a set of query points, high-degree polynomials in the canonical basis in Minterpy become numerically unstable to evaluate and therefore is not recommended.
```

As an example, consider the two-dimensional multi-index set:

$$
A = \{ (0, 0) , (1, 0), (2, 0), (0, 1), (1, 1), (2, 1) \}.
$$

The canonical basis polynomials according to the above multi-index set of exponents are given in the table below.

| $\boldsymbol{\alpha} \in A$ | $\Psi_{\mathrm{can}, \boldsymbol{\alpha} }$ |
|:---------:|:------------:|
| $(0, 0)$  | $1.0$        | 
| $(1, 0)$  | $x_1$        |
| $(2, 0)$  | $x_1^2$      |
| $(0, 1)$  | $x_2$        |
| $(1, 1)$  | $x_1 x_2$    |
| $(2, 1)$  | $x_1^2 x_2$  |

### Chebyshev basis

The multidimensional Chebyshev basis polynomial (of the first kind) associated with the multi-index element $\boldsymbol{\alpha}$ is defined as:

$$
\Psi_{\mathrm{cheb}, \boldsymbol{\alpha}} (\boldsymbol{x}) = T_{\alpha_1} (x_1) \cdots T_{\alpha_m} (x_m) = \prod_{j = 1}^m T_{\alpha_j} (x_j),
$$

where $T_{\alpha_j} (x_j)$ is the $\alpha_j$th-degree (one-dimensional) Chebyshev polynomial of the first kind associated with the $j$th-dimension.

The one-dimensional Chebyshev polynomial of the first kind satisfies the following three-term recurrence (TTR) relation:

$$
\begin{aligned}
  T_0 (x) & = 1 \\
  T_1 (x) & = x \\
  T_{n + 1} (x) & = 2 \, x \, T_n(x)  - T_{n - 1} (x)
\end{aligned}
$$

As an example, consider the two-dimensional multi-index set:

$$
A = \{ (0, 0) , (1, 0), (2, 0), (0, 1), (1, 1) \}.
$$

The Chebyshev basis polynomial (of the first kind) according to the above multi-index set of exponents are given in the table below.

| $\boldsymbol{\alpha} \in A$ | $\Psi_{\mathrm{cheb}, \boldsymbol{\alpha}}$ | Full expression |
|:---------:|:-------:|:--:|
| $(0, 0)$  | $T_0 (x_1) T_0 (x_2)$  | $1.0$         |
| $(1, 0)$  | $T_1 (x_1) T_0 (x_2)$  | $x_1$         |
| $(2, 0)$  | $T_2 (x_1) T_0 (x_2)$  | $2 x_1^2 - 1$ |
| $(0, 1)$  | $T_0 (x_1) T_1 (x_2)$  | $x_2$         |
| $(1, 1)$  | $T_1 (x_1) T_1 (x_2)$  | $x_1 x_2$     |

## Change of basis

Polynomials represented in one basis can be transformed into a polynomial in another basis with the help of various transformation classes listed in the tables below.

| From Lagrange To... | Transformation |
|:------------:|:---:|
| Newton       | {py:class}`LagrangeToNewton <.transformations.lagrange.LagrangeToNewton>` |
| Canonical    | {py:class}`LagrangeToCanonical <.transformations.lagrange.LagrangeToCanonical>` |
| Chebyshev    | {py:class}`LagrangeToChebyshev <.transformations.lagrange.LagrangeToChebyshev>` |
| **From Newton To...** |  |
| Lagrange    | {py:class}`NewtonToLagrange <.transformations.newton.NewtonToLagrange>` |
| Canonical   |{py:class}`NewtonToCanonical <.transformations.newton.NewtonToCanonical>` | 
| Chebyshev   | {py:class}`NewtonToChebyshev <.transformations.newton.NewtonToChebyshev>` |
| **From canonical To...** | |
| Lagrange | {py:class}`CanonicalToLagrange <.transformations.canonical.CanonicalToLagrange>` |
| Newton   | {py:class}`CanonicalToNewton <.transformations.canonical.CanonicalToNewton>` |
| Chebyshev | {py:class}`CanonicalToChebyshev <.transformations.canonical.CanonicalToChebyshev>` |
| **From Chebyshev To...** |  |
| Lagrange | {py:class}`ChebyshevToLagrange <.transformations.chebyshev.ChebyshevToLagrange>` |
| Newton | {py:class}`ChebyshevToNewton <.transformations.chebyshev.ChebyshevToNewton>` |
| Canonical | {py:class}`ChebyshevToCanonical <.transformations.chebyshev.ChebyshevToCanonical>` |

For demonstrantion, see a couple of examples below.

## Example: One-dimensional polynomial

Consider the following one-dimensional polynomial:

$$
Q(x) = 10 + 4.5 x - 3 x^2 + 1.5 x^3 - 1.2 x^4, x \in [-1, 1].
$$

This polynomial is readily expressed in the canonical basis.

First, create the multi-index set of the polynomial:

In [ ]:
mi_1d = mp.MultiIndexSet(np.array([[0], [1], [2], [3], [4]]), lp_degree=1.0)

Then store the coefficients in an array:

In [ ]:
coeffs_1d = np.array([5., 2.5, -2, -5, 1.])

Finally, construct a polynomial in the canonical basis:

In [ ]:
can_poly_1d = mp.CanonicalPolynomial(mi_1d, coeffs_1d)
can_poly_1d

The plot of the polynomial is shown below.

In [ ]:
xx_test = np.linspace(-1, 1, 1000)
plt.plot(xx_test, can_poly_1d(xx_test))
plt.xlabel("$x$", fontsize=14)
plt.ylabel("$y$", fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=12);

The same polynomial can be expressed in a different basis using the help of a transformation class.
Note that all instance of the transformation class must be called to carry out the actual transformation that returns a polynomial in the new basis.

The one-dimensional canonical basis polynomial are shown in the plots below.

In [ ]:
can_monomials_1d = np.prod(
        np.power(
            xx_test[:, np.newaxis][:, None, :],
            can_poly_1d.multi_index.exponents[None, :, :],
        ),
        axis=-1,
    )
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))

axs[0, 0].plot(xx_test, can_monomials_1d[:, 0])
axs[0, 0].set_xlabel("$x$", fontsize=14);
axs[0, 0].set_ylabel("$y$", fontsize=14);
axs[0, 0].set_title("$\\Psi_{\\mathrm{can}, (0)}$", fontsize=14)
axs[0, 0].tick_params(axis='both', which='major', labelsize=12)

axs[0, 1].plot(xx_test, can_monomials_1d[:, 1])
axs[0, 1].set_xlabel("$x$", fontsize=14);
axs[0, 1].set_title("$\\Psi_{\\mathrm{can}, (1)}$", fontsize=14)
axs[0, 1].tick_params(axis='both', which='major', labelsize=12)

axs[0, 2].plot(xx_test, can_monomials_1d[:, 2])
axs[0, 2].set_xlabel("$x$", fontsize=14);
axs[0, 2].set_title("$\\Psi_{\\mathrm{can}, (2)}$", fontsize=14)
axs[0, 2].tick_params(axis='both', which='major', labelsize=12)

axs[1, 0].plot(xx_test, can_monomials_1d[:, 3])
axs[1, 0].set_xlabel("$y$", fontsize=14);
axs[1, 0].set_xlabel("$x$", fontsize=14);
axs[1, 0].set_title("$\\Psi_{\\mathrm{can}, (3)}$", fontsize=14)
axs[1, 0].tick_params(axis='both', which='major', labelsize=12)

axs[1, 1].plot(xx_test, can_monomials_1d[:, 4])
axs[1, 1].set_xlabel("$x$", fontsize=14);
axs[1, 1].set_title("$\\Psi_{\\mathrm{can}, (4)}$", fontsize=14)
axs[1, 1].tick_params(axis='both', which='major', labelsize=12)


axs[1, 2].get_xaxis().set_visible(False)
axs[1, 2].get_yaxis().set_visible(False)
axs[1, 2].spines['top'].set_visible(False)
axs[1, 2].spines['bottom'].set_visible(False)
axs[1, 2].spines['right'].set_visible(False)
axs[1, 2].spines['left'].set_visible(False)

fig.tight_layout()


### In the (1D) Lagrange basis

The transformation of a polynomial in the canonical basis to the one in the Lagrange basis is carried out by an instance of {py:class}`CanonicalToLagrange <.transformations.canonical.CanonicalToLagrange>`:

In [ ]:
lag_poly_1d = mp.CanonicalToLagrange(can_poly_1d)()

The coefficients of the Lagrange polynomial are:

In [ ]:
lag_poly_1d.coeffs

These coefficients are the function values at the unisolvent nodes:

In [ ]:
can_poly_1d(can_poly_1d.grid.unisolvent_nodes)

```{note}
Although the canonical polynomial can be defined without the need for an interpolation grid, the transformation to a Lagrange basis requires such a grid. If not specified during construction, Minterpy automatically creates an interpolating grid based on the Leja-ordered Chebyshev-Lobatto points.
```

The one-dimensional Lagrange basis polynomials are shown in the plots below.

In [ ]:
lag_monomials_newt_basis = mp.NewtonPolynomial(
    lag_poly_1d.multi_index,
    mp.LagrangeToNewton(lag_poly_1d).transformation_operator.array_repr_full,
    grid=lag_poly_1d.grid,
)
lag_monomials_1d = lag_monomials_newt_basis(xx_test)

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))

axs[0, 0].plot(xx_test, lag_monomials_1d[:, 0])
axs[0, 0].set_xlabel("$x$", fontsize=14);
axs[0, 0].set_ylabel("$y$", fontsize=14);
axs[0, 0].set_title("$\\Psi_{\\mathrm{lag}, (0)}$", fontsize=14)
axs[0, 0].tick_params(axis='both', which='major', labelsize=12)

axs[0, 1].plot(xx_test, lag_monomials_1d[:, 1])
axs[0, 1].set_xlabel("$x$", fontsize=14);
axs[0, 1].set_title("$\\Psi_{\\mathrm{lag}, (1)}$", fontsize=14)
axs[0, 1].tick_params(axis='both', which='major', labelsize=12)

axs[0, 2].plot(xx_test, lag_monomials_1d[:, 2])
axs[0, 2].set_xlabel("$x$", fontsize=14);
axs[0, 2].set_title("$\\Psi_{\\mathrm{lag}, (2)}$", fontsize=14)
axs[0, 2].tick_params(axis='both', which='major', labelsize=12)

axs[1, 0].plot(xx_test, lag_monomials_1d[:, 3])
axs[1, 0].set_xlabel("$y$", fontsize=14);
axs[1, 0].set_xlabel("$x$", fontsize=14);
axs[1, 0].set_title("$\\Psi_{\\mathrm{lag}, (3)}$", fontsize=14)
axs[1, 0].tick_params(axis='both', which='major', labelsize=12)

axs[1, 1].plot(xx_test, lag_monomials_1d[:, 4])
axs[1, 1].set_xlabel("$x$", fontsize=14);
axs[1, 1].set_title("$\\Psi_{\\mathrm{lag}, (4)}$", fontsize=14)
axs[1, 1].tick_params(axis='both', which='major', labelsize=12)

axs[1, 2].get_xaxis().set_visible(False)
axs[1, 2].get_yaxis().set_visible(False)
axs[1, 2].spines['top'].set_visible(False)
axs[1, 2].spines['bottom'].set_visible(False)
axs[1, 2].spines['right'].set_visible(False)
axs[1, 2].spines['left'].set_visible(False)

fig.tight_layout()

### In the (1D) Newton basis

The transformation of a polynomial in the canonical basis to the one in the Newton basis is carried out by an instance of {py:class}`CanonicalToNewton <.transformations.canonical.CanonicalToNewton>`:

In [ ]:
nwt_poly_1d = mp.CanonicalToNewton(can_poly_1d)()

The coefficients of the Newton polynomial are:

In [ ]:
nwt_poly_1d.coeffs

Although the canonical polynomial can be defined without the need for an interpolation grid, the transformation to a Newton basis requires such a grid. If not specified during construction, Minterpy automatically creates an interpolating grid based on the Leja-ordered Chebyshev-Lobatto points.
This means the generating points of the grid are:

In [ ]:
nwt_poly_1d.grid.generating_points

With these coefficients and generating points, the same polynomial can be expressed in the Newton basis as follows:

$$
Q(x) = 1.5 - 2.5 (x - 1) - 1 (x - 1) (x + 1) - 4.29289322 (x - 1) (x + 1) x + (x - 1) (x + 1) x (x - 0.707106781).
$$

The one-dimensional Newton basis polynomials are shown in the plots below.

In [ ]:
nwt_monomials_1d = mp.utils.polynomials.newton.eval_newton_monomials(
    xx_test[:, np.newaxis],
    nwt_poly_1d.multi_index.exponents,
    nwt_poly_1d.grid.generating_points,
)
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))

axs[0, 0].plot(xx_test, nwt_monomials_1d[:, 0])
axs[0, 0].set_xlabel("$x$", fontsize=14);
axs[0, 0].set_ylabel("$y$", fontsize=14);
axs[0, 0].set_title("$\\Psi_{\\mathrm{nwt}, (0)}$", fontsize=14)
axs[0, 0].tick_params(axis='both', which='major', labelsize=12)

axs[0, 1].plot(xx_test, nwt_monomials_1d[:, 1])
axs[0, 1].set_xlabel("$x$", fontsize=14);
axs[0, 1].set_title("$\\Psi_{\\mathrm{nwt}, (1)}$", fontsize=14)
axs[0, 1].tick_params(axis='both', which='major', labelsize=12)

axs[0, 2].plot(xx_test, nwt_monomials_1d[:, 2])
axs[0, 2].set_xlabel("$x$", fontsize=14);
axs[0, 2].set_title("$\\Psi_{\\mathrm{nwt}, (2)}$", fontsize=14)
axs[0, 2].tick_params(axis='both', which='major', labelsize=12)

axs[1, 0].plot(xx_test, nwt_monomials_1d[:, 3])
axs[1, 0].set_xlabel("$x$", fontsize=14);
axs[1, 0].set_ylabel("$y$", fontsize=14);
axs[1, 0].set_title("$\\Psi_{\\mathrm{nwt}, (3)}$", fontsize=14)
axs[1, 0].tick_params(axis='both', which='major', labelsize=12)

axs[1, 1].plot(xx_test, nwt_monomials_1d[:, 4])
axs[1, 1].set_xlabel("$x$", fontsize=14);
axs[1, 1].set_title("$\\Psi_{\\mathrm{nwt}, (4)}$", fontsize=14)
axs[1, 1].tick_params(axis='both', which='major', labelsize=12)


axs[1, 2].get_xaxis().set_visible(False)
axs[1, 2].get_yaxis().set_visible(False)
axs[1, 2].spines['top'].set_visible(False)
axs[1, 2].spines['bottom'].set_visible(False)
axs[1, 2].spines['right'].set_visible(False)
axs[1, 2].spines['left'].set_visible(False)

fig.tight_layout()

### In the (1D) Chebyshev basis

The transformation of a polynomial in the canonical basis to the one in the Chebyshev basis is carried out by an instance of {py:class}`CanonicalToChebyshev <.transformations.canonical.CanonicalToChebyshev>`:

In [ ]:
cheb_poly_1d = mp.CanonicalToChebyshev(can_poly_1d)()

The coefficients of the Chebyshev polynomial are:

In [ ]:
cheb_poly_1d.coeffs

With these coefficients and generating points, the same polynomial can be expressed in the Chebyshev basis as follows:

$$
Q(x) = 4.375 - 1.25 T_1 (x) - 0.5 T_2 (x) - 1.25 T_3 (x) + 0.125 T_4(x).
$$

The one-dimensional Chebyshev basis polynomials are shown in the plots below.

In [ ]:
cheb_monomials_1d = mp.utils.polynomials.chebyshev.evaluate_monomials(
    xx_test[:, np.newaxis],
    cheb_poly_1d.multi_index.exponents,
)
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))

axs[0, 0].plot(xx_test, cheb_monomials_1d[:, 0])
axs[0, 0].set_xlabel("$x$", fontsize=14);
axs[0, 0].set_ylabel("$y$", fontsize=14);
axs[0, 0].set_title("$\\Psi_{\\mathrm{cheb}, (0)}$", fontsize=14)
axs[0, 0].tick_params(axis='both', which='major', labelsize=12)

axs[0, 1].plot(xx_test, cheb_monomials_1d[:, 1])
axs[0, 1].set_xlabel("$x$", fontsize=14);
axs[0, 1].set_title("$\\Psi_{\\mathrm{cheb}, (1)}$", fontsize=14)
axs[0, 1].tick_params(axis='both', which='major', labelsize=12)

axs[0, 2].plot(xx_test, cheb_monomials_1d[:, 2])
axs[0, 2].set_xlabel("$x$", fontsize=14);
axs[0, 2].set_title("$\\Psi_{\\mathrm{cheb}, (2)}$", fontsize=14)
axs[0, 2].tick_params(axis='both', which='major', labelsize=12)

axs[1, 0].plot(xx_test, cheb_monomials_1d[:, 3])
axs[1, 0].set_xlabel("$x$", fontsize=14);
axs[1, 0].set_ylabel("$y$", fontsize=14);
axs[1, 0].set_title("$\\Psi_{\\mathrm{cheb}, (3)}$", fontsize=14)
axs[1, 0].tick_params(axis='both', which='major', labelsize=12)

axs[1, 1].plot(xx_test, cheb_monomials_1d[:, 4])
axs[1, 1].set_xlabel("$x$", fontsize=14);
axs[1, 1].set_title("$\\Psi_{\\mathrm{cheb}, (4)}$", fontsize=14)
axs[1, 1].tick_params(axis='both', which='major', labelsize=12)


axs[1, 2].get_xaxis().set_visible(False)
axs[1, 2].get_yaxis().set_visible(False)
axs[1, 2].spines['top'].set_visible(False)
axs[1, 2].spines['bottom'].set_visible(False)
axs[1, 2].spines['right'].set_visible(False)
axs[1, 2].spines['left'].set_visible(False)

fig.tight_layout()

## Example: Two-dimensional polynomial

Consider now the two-dimensional function:

$$
f(x_1, x_2) = \left( 25 - (x_1 - 0.5)^2 - (x_2 - 0.5)^2 \right)^{0.5}, x_1, x_2 \in [-1, 1].
$$

The function is defined in Python as follows:

In [ ]:
def fun(xx):
    return np.sqrt(25 - (xx[:, 0] - 0.5)**2 - (xx[:, 1] - 0.5)**2)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# --- Create 2D data
xx_1d = np.linspace(-1.0, 1.0, 1000)[:, np.newaxis]
mesh_2d = np.meshgrid(xx_1d, xx_1d)
xx_2d = np.array(mesh_2d).T.reshape(-1, 2)
yy_2d = fun(xx_2d)

# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    yy_2d.reshape(1000,1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("Surface plot", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Contour
axs_2 = plt.subplot(122)
cf = axs_2.contourf(
    mesh_2d[0], mesh_2d[1], yy_2d.reshape(1000, 1000).T, cmap="plasma"
)
axs_2.set_xlabel("$x_1$", fontsize=14)
axs_2.set_ylabel("$x_2$", fontsize=14)
axs_2.set_title("Contour plot", fontsize=14)
divider = make_axes_locatable(axs_2)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(cf, cax=cax, orientation='vertical')
axs_2.axis('scaled')
axs_2.tick_params(axis='both', which='major', labelsize=12)

fig.tight_layout(pad=6.0)

Because the function is not a polynomial (a square root is involved), creating a polynomial that approximates the function is much more intuitive in the Lagrange basis. In the other basis, it is not straightforward to intuit what the coefficient values should be.

First, create a two-dimensional multi-index of specified degree:

In [ ]:
mi_2d = mp.MultiIndexSet.from_degree(2, 3, 1.0)
mi_2d

Then, create an interpolation grid and evaluate the function at the unisolvent nodes:

In [ ]:
# Interpolation grid
grd_2d = mp.Grid(mi_2d)
# Coefficients of the Lagrange polynomial
coeffs_2d = grd_2d(fun)

Finally, construct a polynomial in the Lagrange basis given the grid and coefficients:

In [ ]:
lag_poly_2d = mp.LagrangePolynomial.from_grid(grd_2d, coeffs_2d)

The two-dimensional Lagrange basis polynomials are shown in the plots below.

In [ ]:
# --- Create 2D data
xx_1d = np.linspace(-1.0, 1.0, 1000)[:, np.newaxis]
mesh_2d = np.meshgrid(xx_1d, xx_1d)
xx_2d = np.array(mesh_2d).T.reshape(-1, 2)
lag_monomials_newt_basis_2d = mp.NewtonPolynomial(
    lag_poly_2d.multi_index,
    mp.LagrangeToNewton(lag_poly_2d).transformation_operator.array_repr_full,
    grid=lag_poly_2d.grid,
)
lag_monomials_2d = lag_monomials_newt_basis_2d(xx_2d)

# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    lag_monomials_2d[:, 0].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{lag}, (0, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    lag_monomials_2d[:, 1].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{lag}, (1, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    lag_monomials_2d[:, 2].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{lag}, (2, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    lag_monomials_2d[:, 3].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{lag}, (3, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    lag_monomials_2d[:, 4].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{lag}, (0, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    lag_monomials_2d[:, 5].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{lag}, (1, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    lag_monomials_2d[:, 6].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{lag}, (2, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    lag_monomials_2d[:, 7].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{lag}, (0, 2)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    lag_monomials_2d[:, 8].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{lag}, (1, 2)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    lag_monomials_2d[:, 9].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{lag}, (0, 3)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

### In the (2D) Newton basis

You can transform a Minterpy polynomial in the Lagrange basis to another polynomial in the Newton with the help of {py:class}`LagrangeToNewton <.transformations.lagrange.LagrangeToNewton>`:

In [ ]:
nwt_poly_2d = mp.LagrangeToNewton(lag_poly_2d)()

The coefficients of the Newton polynomial are:

In [ ]:
nwt_poly_2d.coeffs

The two-dimensional Newton basis polynomials are shown in the plots below.

In [ ]:
# --- Create 2D data
xx_1d = np.linspace(-1.0, 1.0, 1000)[:, np.newaxis]
mesh_2d = np.meshgrid(xx_1d, xx_1d)
xx_2d = np.array(mesh_2d).T.reshape(-1, 2)
nwt_monomials_2d = mp.utils.polynomials.newton.eval_newton_monomials(
    xx_2d,
    mi_2d.exponents,
    grd_2d.generating_points,
)

# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    nwt_monomials_2d[:, 0].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{nwt}, (0, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    nwt_monomials_2d[:, 1].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{nwt}, (1, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    nwt_monomials_2d[:, 2].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{nwt}, (2, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    nwt_monomials_2d[:, 3].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{nwt}, (3, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    nwt_monomials_2d[:, 4].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{nwt}, (0, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    nwt_monomials_2d[:, 5].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{nwt}, (1, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    nwt_monomials_2d[:, 6].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{nwt}, (2, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    nwt_monomials_2d[:, 7].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{nwt}, (0, 2)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    nwt_monomials_2d[:, 8].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{nwt}, (1, 2)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    nwt_monomials_2d[:, 9].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{nwt}, (0, 3)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

### In the (2D) canonical basis

You can transform a Minterpy polynomial in the Lagrange basis to another polynomial in the canonical basis with the help of {py:class}`LagrangeToCanonical <.transformations.lagrange.LagrangeToCanonical>`:

In [ ]:
can_poly_2d = mp.LagrangeToCanonical(lag_poly_2d)()

The coefficients of the canonical polynomial are:

In [ ]:
can_poly_2d.coeffs

The two-dimensional canonical basis polynomials are shown in the plots below.

In [ ]:
# --- Create 2D data
xx_1d = np.linspace(-1.0, 1.0, 1000)[:, np.newaxis]
mesh_2d = np.meshgrid(xx_1d, xx_1d)
xx_2d = np.array(mesh_2d).T.reshape(-1, 2)
can_monomials_2d = np.prod(
        np.power(
            xx_2d[:, None, :],
            can_poly_2d.multi_index.exponents[None, :, :],
        ),
        axis=-1,
    )

# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    can_monomials_2d[:, 0].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{can}, (0, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    can_monomials_2d[:, 1].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{can}, (1, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    can_monomials_2d[:, 2].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{can}, (2, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    can_monomials_2d[:, 3].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{can}, (3, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    can_monomials_2d[:, 4].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{can}, (0, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    can_monomials_2d[:, 5].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{can}, (1, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    can_monomials_2d[:, 6].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{can}, (2, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    can_monomials_2d[:, 7].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{can}, (0, 2)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    can_monomials_2d[:, 8].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{can}, (1, 2)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    can_monomials_2d[:, 9].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{can}, (0, 3)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

### In the (2D) Chebyshev basis

Finally, you can transform a Minterpy polynomial in the Lagrange basis to another polynomial in the Chebyshev basis (of the first kind) with the help of {py:class}`LagrangeToChebyshev <.transformations.lagrange.LagrangeToChebyshev>`:

In [ ]:
cheb_poly_2d = mp.LagrangeToChebyshev(lag_poly_2d)()

The coefficients of the canonical polynomial are:

In [ ]:
cheb_poly_2d.coeffs

The two-dimensional Chebyshev basis polynomials (of the first kind) are shown in the plots below.

In [ ]:
# --- Create 2D data
xx_1d = np.linspace(-1.0, 1.0, 1000)[:, np.newaxis]
mesh_2d = np.meshgrid(xx_1d, xx_1d)
xx_2d = np.array(mesh_2d).T.reshape(-1, 2)
cheb_monomials_2d = mp.utils.polynomials.chebyshev.evaluate_monomials(
    xx_2d,
    cheb_poly_2d.multi_index.exponents,
)

# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    cheb_monomials_2d[:, 0].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{cheb}, (0, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    cheb_monomials_2d[:, 1].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{cheb}, (1, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    cheb_monomials_2d[:, 2].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{cheb}, (2, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    cheb_monomials_2d[:, 3].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{cheb}, (3, 0)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    cheb_monomials_2d[:, 4].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{cheb}, (0, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    cheb_monomials_2d[:, 5].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{cheb}, (1, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    cheb_monomials_2d[:, 6].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{cheb}, (2, 1)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    cheb_monomials_2d[:, 7].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{cheb}, (0, 2)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    cheb_monomials_2d[:, 8].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{cheb}, (1, 2)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Surface
axs_1 = plt.subplot(122, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    cheb_monomials_2d[:, 9].reshape(1000, 1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("$\\Psi_{\\mathrm{cheb}, (0, 3)}$", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

## From interpolant to interpolating polynomial

The {doc}`Quickstart Guide </getting-started/functions-approximation>` introduced
you with the {py:func}`interpolate() <.interpolation.interpolate>` function to
conveniently create an interpolant out of a function for a given spatial
dimension, polynomial degree, and $l_p$-degree.
As noted in the previous tutorials, while {py:func}`interpolate() <.interpolation.interpolate>`
does not produce an instance of Minterpy polynomials
(and instead, an instance of {py:class}`Interpolant <.interpolation.Interpolant>`)
you can conveniently retrieve the polynomial by the methods summarized in the
table below.

|                               Method                                |                    Return the interpolant as a polynomial in the...                     |
|:-------------------------------------------------------------------:|:---------------------------------------------------------------------------------------:|
|    {py:meth}`to_newton() <.interpolation.Interpolant.to_newton>`    |     {py:class}`NewtonPolynomial <.polynomials.newton_polynomial.NewtonPolynomial>`      |
|  {py:meth}`to_lagrange() <.interpolation.Interpolant.to_lagrange>`  |  {py:class}`LagrangePolynomial <.polynomials.lagrange_polynomial.LagrangePolynomial>`   |
| {py:meth}`to_canonical() <.interpolation.Interpolant.to_canonical>` | {py:class}`CanonicalPolynomial <.polynomials.canonical_polynomial.CanonicalPolynomial>` |
| {py:meth}`to_chebyshev() <.interpolation.Interpolant.to_chebyshev>` | {py:class}`CanonicalPolynomial <.polynomials.chebyshev_polynomial.ChebyshevPolynomial>` |

## Summary

In this tutorial, you've explored the different polynomial bases supported by Minterpy. Depending on the problem at hand, it may be more intuitive to construct a polynomial in a specific basis.

Once a polynomial has been constructed in one basis, it can be transformed into another. The result of this transformation is a polynomial that is equivalent to the original but represented in a different basis.

Note, however, that Minterpy has different supported features for each polynomial basis as summarized by the following table.

| Operations                        | Lagrange | Newton   | Canonical | Chebyshev |
|:----------------------------------|:--------:|:--------:|:---------:|:---------:|
| Transformation from-and-to        | &#10003; | &#10003; | &#10003;  | &#10003;  |
| Evaluation                        | &#10005; | &#10003; | &#10003;  | &#10003;  |
| Addition/Subtraction (Scalar)     | &#10003; | &#10003; | &#10003;  | &#10003;  |
| Addition/Subtraction (Polynomial) | &#10005; | &#10003; | &#10003;  | &#10003;  |
| Multiplication (Scalar)           | &#10003; | &#10003; | &#10003;  | &#10003;  |
| Multiplication (Polynomial)       | &#10005; | &#10003; | &#10003;  | &#10003;  |
| Division (Scalar)                 | &#10003; | &#10003; | &#10003;  | &#10003;  |
| Division (Polynomial)             | &#10005; | &#10005; | &#10005;  | &#10005;  |
| Differentiation                   | &#10005; | &#10003; | &#10003;  | &#10005;  |
| Definite integration              | &#10005; | &#10003; | &#10003;  | &#10005;  |

---

So far, you've learned:

- how to construct Minterpy polynomials in one or more dimensions
- what operations you can perform with Minterpy polynomials (arithmetic and calculus operations)
- the different polynomial bases and how to transform between them

In all these tutorials, you typically start with a given function of interest, where you can evaluate it at chosen locations (i.e., the unisolvent nodes) to construct an interpolating polynomial.

One important problem remains: _How do you construct a polynomial from a given set of scattered data?_